In [9]:
import os
import math
import random
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset
from PIL import Image
import requests
from io import BytesIO

# Project modules (adjust import paths if needed in your repo)
from imports.configs.config import load_config, set_global_seed
from imports.encoders import VisionEncoder, AudioEncoder
from imports.perceiver import PerceiverLatentEncoder, ProjectorMLP
from imports.align_training.text_encoder import HFTextEncoder, HFTextEncoderConfig
from imports.align_training.losses import matryoshka_contrastive_loss, clip_contrastive_loss


In [10]:
cfg = load_config("imports/configs/config.yaml")
cfg.resolve_device_and_dtype()

device = cfg.torch_device
dtype = cfg.torch_dtype

print("Device:", device)
print("Dtype:", dtype)

set_global_seed(cfg.misc.seed)


Device: cuda
Dtype: torch.float32


🖼️ 1. Hugging Face Datasets (PixMo-Cap & LibriSpeech)
🧱 Cell 1.1 – Helper: load image from URL

In [47]:
import requests
from requests.exceptions import TooManyRedirects, RequestException
from PIL import Image
from io import BytesIO

def load_image_from_url(url: str, timeout: float = 10.0) -> Image.Image:
    """
    Robust image loader:
      - Tries to fetch the URL.
      - On TooManyRedirects / HTTP error, returns a dummy image instead
        of crashing the DataLoader.
    """
    try:
        resp = requests.get(url, timeout=timeout)
        resp.raise_for_status()
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        return img
    except TooManyRedirects:
        print(f"[WARN] Too many redirects for URL, using dummy image:\n  {url}")
    except RequestException as e:
        print(f"[WARN] Request failed for URL ({e}), using dummy image:\n  {url}")
    except Exception as e:
        print(f"[WARN] Unexpected error loading URL ({e}), using dummy image:\n  {url}")

    # Fallback: 224x224 gray image (CLIP-friendly size)
    return Image.new("RGB", (224, 224), color=(128, 128, 128))


In [12]:
# allenai/pixmo-cap
pixmo_raw = load_dataset("allenai/pixmo-cap", split="train")
print(pixmo_raw)
print("Sample:", {k: pixmo_raw[0][k] for k in ["image_url", "caption"]})


Dataset({
    features: ['image_url', 'caption', 'transcripts'],
    num_rows: 717042
})
Sample: {'image_url': 'https://pixmo.s3.us-west-2.amazonaws.com/birds/1491.png', 'caption': "This photograph depicts a striking black bird, possibly a grackle or similar species, perched on a white cement wall with red stains. The bird's sleek, elongated body is adorned with iridescent feathers that shimmer with shades of blue, purple, and green, most prominently on its wings and back. Its piercing yellow eye and long, sharp beak lend it a fierce, almost aerodynamic appearance. The bird's dark, slender legs and short talons grip the edge of the cement structure, which resembles a divider or barrier, possibly in an outdoor setting like a park or building patio. The background is a blur of green and white hues, hinting at lush tropical plants and tall trees, setting a serene, natural scene. The bird is poised, looking towards the upper right-hand corner of the frame, with its long tail feathers trail

In [18]:
class PixmoHFAlignmentDataset(Dataset):
    """
    Wraps allenai/pixmo-cap into:
        {
            "image": PIL.Image,
            "text": str,
        }
    """

    def __init__(self, hf_ds, max_samples: int | None = None):
        if max_samples is not None:
            max_samples = min(max_samples, len(hf_ds))
            hf_ds = hf_ds.select(range(max_samples))
        self.hf_ds = hf_ds

    def __len__(self) -> int:
        return len(self.hf_ds)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        row = self.hf_ds[idx]
        img = load_image_from_url(row["image_url"])
        caption = row["caption"]
        return {
            "image": img,
            "text": caption,
        }


In [36]:
from datasets import load_dataset
from torch.utils.data import Dataset
from typing import Dict, Any, List
import io
import librosa
import numpy as np

print("\nLoading LibriSpeech ASR in streaming mode...")

librispeech_raw = load_dataset(
    "librispeech_asr",
    "all",
    streaming=True,
    split="train.other.500",
)



Loading LibriSpeech ASR in streaming mode...


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:

print("Loaded streaming dataset:", librispeech_raw)


class LibriSpeechHFAlignmentDataset(Dataset):
    """
    Wraps a list of preprocessed LibriSpeech examples into:
        {
            "waveform": 1D float32 numpy array,
            "sampling_rate": int,
            "text": str,
        }
    """

    def __init__(self, samples: List[Dict[str, Any]]):
        # 'samples' is a plain Python list of dicts
        self.samples = samples

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        return self.samples[idx]




Loading LibriSpeech ASR in streaming mode...


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Loaded streaming dataset: IterableDataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_shards: 64
})


In [26]:
class LibriSpeechHFAlignmentDataset(Dataset):
    """
    Wraps openslr/librispeech_asr into:
        {
            "waveform": 1D float32 numpy array,
            "sampling_rate": int,
            "text": str,
        }
    """

    def __init__(self, hf_ds, max_samples: int | None = None):
        if max_samples is not None:
            max_samples = min(max_samples, len(hf_ds))
            hf_ds = hf_ds.select(range(max_samples))
        self.hf_ds = hf_ds

    def __len__(self) -> int:
        return len(self.hf_ds)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        row = self.hf_ds[idx]
        audio = row["audio"]  # dict: {"array": np.ndarray, "sampling_rate": int}
        wav = audio["array"].astype("float32")
        sr = int(audio["sampling_rate"])
        text = row["text"]
        return {
            "waveform": wav,
            "sampling_rate": sr,
            "text": text,
        }


In [38]:

# Disable automatic decoding → we want raw bytes
audio_stream = librispeech_raw.decode(False)

max_samples = 5000
subset = []

print(f"\nTaking up to {max_samples} examples in streaming mode...")

for ex in audio_stream:
    subset.append(ex)
    if len(subset) >= max_samples:
        break

print("\nSubset collected:", len(subset))
if len(subset) > 0:
    print("Keys:", subset[0].keys())
    print("Example 0 (truncated):", {k: subset[0][k] for k in subset[0].keys() if k != "audio"})

# Helper ----------------------------------------------------------

def load_waveform_from_streaming_example(example, target_sr=16_000):
    """
    Convert LibriSpeech streaming example -> mono waveform @ target_sr.
    Expects example["audio"]["bytes"] to be present.
    """
    audio_info = example["audio"]
    audio_bytes = audio_info["bytes"]
    if audio_bytes is None:
        raise ValueError("No audio bytes in example.")

    audio_file = io.BytesIO(audio_bytes)
    wav, sr = librosa.load(audio_file, sr=target_sr, mono=True)
    return wav, sr

def compute_duration(wav, sr):
    return len(wav) / float(sr)


# Preprocess into alignment-friendly samples ----------------------

filtered = []

target_sr = 16_000  # assuming cfg is defined earlier

for ex in subset:
    try:
        wav, sr = load_waveform_from_streaming_example(ex, target_sr=target_sr)
    except Exception as e:
        # Optionally log and skip problematic examples
        # log_path = cfg.root_dir / "decode_skipped.log"
        # with open(log_path, "a") as f:
        #     f.write(f"decode_error\ttext={ex.get('text', '')[:80]!r}\terror={repr(e)}\n")
        continue

    # (Optional) filter by duration, e.g., 1s–20s
    dur = compute_duration(wav, sr)
    if not (1.0 <= dur <= 20.0):
        continue

    filtered.append(
        {
            "waveform": wav.astype("float32"),
            "sampling_rate": int(sr),
            "text": ex["text"],
        }
    )

print("Filtered valid Libri examples:", len(filtered))



Taking up to 5000 examples in streaming mode...

Subset collected: 5000
Keys: dict_keys(['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'])
Example 0 (truncated): {'file': '/home/albert/.cache/huggingface/datasets/downloads/extracted/52b35db43bb19b11fc0e8d082c86dd8d065bab1bf6605e6bb5b44b233b44cb51/8296-266250-0000.flac', 'text': 'THE ORDINARY DUTIES OF LIFE MISTER DALY ANXIOUS TO MAKE SOME RETURN FOR THE KINDNESS SHOWN HIM OFFERED TO ACT AS TUTOR TO ALL THE CHILDREN WHO WERE OLD ENOUGH FOR SCHOOL DUTIES', 'speaker_id': 8296, 'chapter_id': 266250, 'id': '8296-266250-0000'}
Filtered valid Libri examples: 4999


In [39]:
# Limits from cfg -------------------------------------------------

MAX_PIXMO_SAMPLES = getattr(cfg.datasets, "pixmo_max_samples", 5000)
MAX_LIBRI_SAMPLES = getattr(cfg.datasets, "librispeech_max_samples", 5000)

# Pixmo still uses the HF non-streaming dataset
pixmo_ds = PixmoHFAlignmentDataset(pixmo_raw, max_samples=MAX_PIXMO_SAMPLES)

# For Libri, we now pass the filtered list and slice it in Python (no .select)
librispeech_ds = LibriSpeechHFAlignmentDataset(filtered[:MAX_LIBRI_SAMPLES])

print("PixMo samples:", len(pixmo_ds))
print("LibriSpeech samples:", len(librispeech_ds))

PixMo samples: 5000
LibriSpeech samples: 4999


In [40]:
def collate_vision(batch: List[Dict[str, Any]]) -> Dict[str, Any]:
    images = [b["image"] for b in batch]
    texts  = [b["text"] for b in batch]
    return {
        "images": images,
        "texts":  texts,
    }


def collate_audio(batch: List[Dict[str, Any]]) -> Dict[str, Any]:
    waveforms      = [b["waveform"] for b in batch]
    sampling_rates = [b["sampling_rate"] for b in batch]
    texts          = [b["text"] for b in batch]
    return {
        "waveforms":      waveforms,
        "sampling_rates": sampling_rates,
        "texts":          texts,
    }


In [42]:
BATCH_SIZE = cfg.training.batch_size

pixmo_loader = DataLoader(
    pixmo_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_vision,
)

librispeech_loader = DataLoader(
    librispeech_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_audio,
)

len(pixmo_loader), len(librispeech_loader)


(313, 313)

## Encoders

In [43]:
vision_enc = VisionEncoder(
    model_name=cfg.models.vision_model_name,  # "openai/clip-vit-base-patch32"
    device=device,
    dtype=dtype,
)
audio_enc = AudioEncoder(
    model_name=cfg.models.audio_model_name,   # "openai/whisper-base"
    device=device,
    dtype=dtype,
    target_sampling_rate=16_000,
)

for p in vision_enc.parameters():
    p.requires_grad = False
for p in audio_enc.parameters():
    p.requires_grad = False

vision_enc.eval()
audio_enc.eval()

print("Vision encoder:", cfg.models.vision_model_name)
print("Audio encoder :", cfg.models.audio_model_name)


Vision encoder: openai/clip-vit-base-patch32
Audio encoder : openai/whisper-base


In [44]:
TEXT_ENCODER_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # or add to config

text_cfg = HFTextEncoderConfig(
    model_name=TEXT_ENCODER_NAME,
    max_length=128,
    trainable=False,
)
text_enc = HFTextEncoder(text_cfg, device=device)
d_text = text_enc.hidden_size
print("Text encoder:", TEXT_ENCODER_NAME)
print("Text hidden size:", d_text)

def text_embed_fn(texts: List[str], max_length: int = 128) -> torch.Tensor:
    with torch.no_grad():
        return text_enc.encode(texts)  # (B, d_text)


Text encoder: sentence-transformers/all-MiniLM-L6-v2
Text hidden size: 384


### Alignment Layers

In [45]:
def masked_mean(x: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
    """
    x: (B, T, D), mask: (B, T) bool
    returns: (B, D)
    """
    mask = mask.to(x.dtype)
    denom = mask.sum(dim=1, keepdim=True).clamp(min=1.0)
    return (x * mask.unsqueeze(-1)).sum(dim=1) / denom


In [49]:
# Probe CLIP feature dim
sample_vision_batch = next(iter(pixmo_loader))
with torch.no_grad():
    v_out = vision_enc.encode_images(sample_vision_batch["images"])
    d_vision = vision_enc.feat_dim
print("Vision feat dim:", d_vision)

[WARN] Request failed for URL (429 Client Error: Unknown Error for url: https://i.imgur.com/ZWBu0oI.jpg), using dummy image:
  https://i.imgur.com/ZWBu0oI.jpg
[WARN] Request failed for URL (404 Client Error: Not Found for url: https://www.libertyflagpoles.com/cdn/shop/articles/MinutementFlagpolesInc.DBALIbertyFlagpoles-237633-Flagpole-Inclement-Weather-Blogbanner1_600x600_crop_center.jpg?v=1691778269), using dummy image:
  https://www.libertyflagpoles.com/cdn/shop/articles/MinutementFlagpolesInc.DBALIbertyFlagpoles-237633-Flagpole-Inclement-Weather-Blogbanner1_600x600_crop_center.jpg?v=1691778269
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: http://i.imgur.com/Kedmm.jpg), using dummy image:
  http://i.imgur.com/Kedmm.jpg
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: http://i.imgur.com/XTYFe.jpg), using dummy image:
  http://i.imgur.com/XTYFe.jpg
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: https://i.imgur.co

ValueError: You have to specify input_ids

In [ ]:
# Probe Whisper feature dim
sample_audio_batch = next(iter(librispeech_loader))
with torch.no_grad():
    a_out = audio_enc.encode_waveforms(
        sample_audio_batch["waveforms"],
        sample_audio_batch["sampling_rates"],
    )
    d_audio = audio_enc.feat_dim
print("Audio feat dim:", d_audio)

In [50]:
# Shared Perceiver dim
d_shared = cfg.architecture.perceiver_dim or d_vision
d_align = d_text

print("Shared (Perceiver) dim:", d_shared)
print("Aligned dim:", d_align)

# Adapters
vision_adapter = nn.Linear(d_vision, d_shared)
audio_adapter  = nn.Linear(d_audio,  d_shared)

NameError: name 'd_vision' is not defined

In [ ]:
# Perceiver
perceiver = PerceiverLatentEncoder(
    d_input=d_shared,
    d_latent=d_shared,
    num_latents=cfg.architecture.num_latents,
    num_layers=cfg.architecture.num_perceiver_layers,
    num_heads=cfg.architecture.num_attn_heads,
    mlp_ratio=cfg.architecture.mlp_ratio,
)

# Projector
projector = ProjectorMLP(
    d_in=d_shared,
    d_hidden=int(cfg.architecture.mlp_ratio * d_shared),
    d_out=d_align,
    dropout=0.1,
)

modules = nn.ModuleDict(
    dict(
        vision_adapter=vision_adapter,
        audio_adapter=audio_adapter,
        perceiver=perceiver,
        projector=projector,
    )
).to(device)

sum(p.numel() for p in modules.parameters() if p.requires_grad)


[WARN] Request failed for URL (429 Client Error: Unknown Error for url: https://i.imgur.com/DpNz9o3.jpg), using dummy image:
  https://i.imgur.com/DpNz9o3.jpg
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: https://i.imgur.com/zm4sBfg.jpg), using dummy image:
  https://i.imgur.com/zm4sBfg.jpg
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: http://i.imgur.com/94MhWU6.jpg), using dummy image:
  http://i.imgur.com/94MhWU6.jpg
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: https://i.imgur.com/00A6s2u.jpg), using dummy image:
  https://i.imgur.com/00A6s2u.jpg
[WARN] Request failed for URL (403 Client Error: Forbidden for url: https://ih1.redbubble.net/image.662143812.9372/mwo,x1000,ipad_2_skin-pad,750x1000,f8f8f8.u10.jpg), using dummy image:
  https://ih1.redbubble.net/image.662143812.9372/mwo,x1000,ipad_2_skin-pad,750x1000,f8f8f8.u10.jpg
[WARN] Request failed for URL (429 Client Error: Unknown Error for url: http://i.

ValueError: You have to specify input_ids

### 🎯 5. Forward passes & Losses (MRL + CLIP)

In [ ]:
@torch.no_grad()
def encode_modality_batch(
    modality: str,
    batch: Dict[str, Any],
) -> Tuple[torch.Tensor, List[str]]:
    """
    Returns:
        z_mod: (B, d_align)
        texts: list[str]
    """
    if modality == "vision":
        images = batch["images"]
        texts  = batch["texts"]

        enc_out = vision_enc.encode_images(images)
        feats = enc_out["feats"]       # (B, T_v, d_vision)
        mask  = enc_out["mask"]        # (B, T_v)

        feats = feats.to(device)
        mask  = mask.to(device)
        h_shared = modules["vision_adapter"](feats)  # (B, T_v, d_shared)

    elif modality == "audio":
        wavs = batch["waveforms"]
        srs  = batch["sampling_rates"]
        texts = batch["texts"]

        enc_out = audio_enc.encode_waveforms(wavs, srs)
        feats = enc_out["feats"]       # (B, T_a, d_audio)
        mask  = enc_out["mask"]        # (B, T_a)

        feats = feats.to(device)
        mask  = mask.to(device)
        h_shared = modules["audio_adapter"](feats)   # (B, T_a, d_shared)

    else:
        raise ValueError(f"Unknown modality: {modality}")

    latents = modules["perceiver"](h_shared, mask)      # (B, L, d_shared)
    z_tokens = modules["projector"](latents)            # (B, L, d_align)
    z_mod = z_tokens.mean(dim=1)                        # (B, d_align)

    return z_mod, texts


In [ ]:
@dataclass
class AlignLossConfig:
    mrl_dims: Tuple[int, ...]
    mrl_temp: float = cfg.mrl.mrl_temp
    mrl_weight: float = cfg.mrl.mrl_weight

    clip_temp: float = 0.07
    clip_weight: float = 1.0

    max_text_length: int = 128

align_cfg = AlignLossConfig(
    mrl_dims=tuple(cfg.mrl.mrl_dims),
)
align_cfg


AlignLossConfig(mrl_dims=(1024, 512, 256), mrl_temp=0.07, mrl_weight=1.0, clip_temp=0.07, clip_weight=1.0, max_text_length=128)

In [ ]:
def forward_align_step(
    modality: str,
    batch: Dict[str, Any],
) -> Tuple[torch.Tensor, Dict[str, float]]:
    """
    Compute total loss + metrics for one mini-batch of a given modality.
    """
    modules.train()  # we only train adapters + perceiver + projector

    # 1) Modality embeddings
    with torch.no_grad():
        z_mod, texts = encode_modality_batch(modality, batch)  # (B, d_align), list[str]

    # 2) Text embeddings
    z_txt = text_embed_fn(texts, max_length=align_cfg.max_text_length)  # (B, d_align)

    # float32 for stability in loss
    z_mod = z_mod.to(torch.float32)
    z_txt = z_txt.to(torch.float32)

    # 3) MRL loss
    loss_mrl = matryoshka_contrastive_loss(
        z_mod,
        z_txt,
        radii=align_cfg.mrl_dims,
        temperature=align_cfg.mrl_temp,
        symmetric=True,
    )

    # 4) CLIP loss
    loss_clip = clip_contrastive_loss(
        z_mod,
        z_txt,
        temperature=align_cfg.clip_temp,
        symmetric=True,
    )

    loss = align_cfg.mrl_weight * loss_mrl + align_cfg.clip_weight * loss_clip

    metrics = {
        f"{modality}/mrl_loss": float(loss_mrl.detach().cpu().item()),
        f"{modality}/clip_loss": float(loss_clip.detach().cpu().item()),
        f"{modality}/total_loss": float(loss.detach().cpu().item()),
    }
    return loss, metrics


In [ ]:
params = list(modules.parameters())

optimizer = torch.optim.AdamW(
    params,
    lr=float(cfg.training.learning_rate),   # "3e-4" in YAML -> cast to float
    weight_decay=cfg.training.weight_decay,
)

print("Trainable params:",
      sum(p.numel() for p in params if p.requires_grad))


In [ ]:
use_wandb = getattr(cfg.misc, "use_wandb", False)

if use_wandb:
    import wandb
    wandb.init(
        project=cfg.misc.wandb_project,
        name=cfg.misc.wandb_run_name,
        config={"align_cfg": align_cfg.__dict__},
    )
else:
    wandb = None


In [ ]:
NUM_EPOCHS = cfg.training.num_epochs
LOG_EVERY = cfg.training.log_every_steps
MAX_GRAD_NORM = cfg.training.max_grad_norm

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\n===== Epoch {epoch}/{NUM_EPOCHS} =====")

    # ---- Vision (PixMo) ----
    for step, batch in enumerate(pixmo_loader, start=1):
        optimizer.zero_grad(set_to_none=True)
        loss, metrics = forward_align_step("vision", batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(modules.parameters(), MAX_GRAD_NORM)
        optimizer.step()

        if step % LOG_EVERY == 0:
            msg = f"[Vision] step {step:05d} | loss={metrics['vision/total_loss']:.4f}"
            print(msg)
            if wandb is not None:
                wandb.log({"epoch": epoch, **metrics})

    # ---- Audio (LibriSpeech) ----
    for step, batch in enumerate(librispeech_loader, start=1):
        optimizer.zero_grad(set_to_none=True)
        loss, metrics = forward_align_step("audio", batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(modules.parameters(), MAX_GRAD_NORM)
        optimizer.step()

        if step % LOG_EVERY == 0:
            msg = f"[Audio ] step {step:05d} | loss={metrics['audio/total_loss']:.4f}"
            print(msg)
            if wandb is not None:
                wandb.log({"epoch": epoch, **metrics})


In [ ]:
os.makedirs("checkpoints", exist_ok=True)
ckpt_path = "checkpoints/alignment_clip_whisper_live.pt"

torch.save(
    {
        "modules": modules.state_dict(),
        "meta": {
            "d_shared": d_shared,
            "d_align": d_align,
            "vision_model_name": cfg.models.vision_model_name,
            "audio_model_name": cfg.models.audio_model_name,
            "text_model_name": TEXT_ENCODER_NAME,
            "mrl_dims": list(align_cfg.mrl_dims),
        },
    },
    ckpt_path,
)
print("Saved alignment checkpoint to:", ckpt_path)


In [ ]:
modules.eval()

# Image example
ex_pix = pixmo_ds[0]
with torch.no_grad():
    z_img, _ = encode_modality_batch(
        "vision",
        {"images": [ex_pix["image"]], "texts": [ex_pix["text"]]},
    )
print("Image embedding shape:", z_img.shape)

# Audio example
ex_aud = librispeech_ds[0]
with torch.no_grad():
    z_aud, _ = encode_modality_batch(
        "audio",
        {
            "waveforms":      [ex_aud["waveform"]],
            "sampling_rates": [ex_aud["sampling_rate"]],
            "texts":          [ex_aud["text"]],
        },
    )
print("Audio embedding shape:", z_aud.shape)
